In [1]:
!pip install torch torchvision tqdm

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.7/23.7 MB 21.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 823.6/823.6 kB 49.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.1/14.1 MB 22.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 731.7/731.7 MB 1.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 410.6/410.6 MB 1.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 121.6/121.6 MB 1.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.5/56.5 MB 1.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 124.2/124.2 MB 1.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 196.0/196.0 MB 1.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 166.0/166.0 MB 1.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 99.1/99.1 kB 1.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 2.7 MB/s eta 0:00:00


In [2]:
from google.colab import drive
drive.mount("/content/drive", force_remount=True)

Mounted at /content/drive


In [3]:
import torch
from torch import nn
from tqdm.auto import tqdm
from torchvision import transforms
from torchvision.utils import make_grid
from torch.utils.data import DataLoader
import matplotlib.pyplot as plt
torch.manual_seed(123)



In [6]:
import os
from PIL import Image

In [5]:
!unzip '/content/drive/MyDrive/Magisterka/Twarz/twarz_1.zip' -d '/content/dataset/'



Archive:  /content/drive/MyDrive/Magisterka/Twarz/twarz_1.zip
  inflating: /content/dataset/621.jpg  
  inflating: /content/dataset/622.jpg  
  inflating: /content/dataset/623.jpg  
  inflating: /content/dataset/624.jpg  
  inflating: /content/dataset/625.jpg  
  inflating: /content/dataset/626.jpg  
  inflating: /content/dataset/627.jpg  
  inflating: /content/dataset/628.jpg  
  inflating: /content/dataset/629.jpg  
  inflating: /content/dataset/630.jpg  
  inflating: /content/dataset/631.jpg  
  inflating: /content/dataset/632.jpg  
  inflating: /content/dataset/633.jpg  
  inflating: /content/dataset/634.jpg  
  inflating: /content/dataset/0.jpg  
  inflating: /content/dataset/1.jpg  
  inflating: /content/dataset/2.jpg  
  inflating: /content/dataset/3.jpg  
  inflating: /content/dataset/4.jpg  
  inflating: /content/dataset/5.jpg  
  inflating: /content/dataset/6.jpg  
  inflating: /content/dataset/7.jpg  
  inflating: /content/dataset/8.jpg  
  inflating: /content/dataset/9.jpg 

In [ ]:
folder_path = '/content/dataset'
image_files = os.listdir(folder_path)

images = []
for image_file in image_files:
    if image_file.endswith('.jpg') or image_file.endswith('.png'):
        image_path = os.path.join(folder_path, image_file)
        image = Image.open(image_path)
        images.append(image)

In [ ]:



sample_img[0].shape